<a href="https://colab.research.google.com/github/jesvin1/Options/blob/master/Option_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>